In [55]:
import pandas as pd
import requests
from pprint import pprint
import datetime
import json
import requests

In [2]:
from config import GOOGLE
from config import OWM

In [18]:
Overview=pd.read_csv("Overview.csv")
GP_list_all=Overview["AreaName"]
GP_list=GP_list_all.unique()
Final_GP_List=GP_list[2:len(GP_list)]
Asthma_data=Overview.loc[Overview['Indicator Name'] == "Asthma: QOF prevalence (6+ yrs)"][["AreaName","Count","Value"]]
Percentage=Asthma_data["Value"][2:len(GP_list)]

In [19]:
api_key=GOOGLE

In [20]:
gp_lat=[]
gp_lng=[]

url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"

lat="52.478667"
long="-1.900024"

for gp in Final_GP_List:
    params={"key":api_key,
            'query':gp,
            'location':f"{lat},{long}",
            'radius':15000,
            'type':'health'
            }
    
    temp = requests.get(url, params=params).json()
    
    try:
        gp_lat.append(temp['results'][0]['geometry']['location']['lat'])
        gp_lng.append(temp['results'][0]['geometry']['location']['lng'])
    except IndexError:
        
        try:
            gp_lat.append(temp['results'][0]['geometry']['location']['lat'])
            gp_lng.append(temp['results'][0]['geometry']['location']['lng'])
            
            params={"key":api_key,
                'query':gp,
                'location':f"{lat},{long}",
                'radius':15000,
                'type':'doctor'
                }

            temp = requests.get(url, params=params).json()
        except IndexError:
            gp_lat.append("N/A")
            gp_lng.append("N/A")
    


In [21]:
df=pd.DataFrame(zip(Final_GP_List,gp_lat,gp_lng,Percentage),columns=["GP Practice","Latitude","Longitude","Percentage"])
df.drop(gp_lng.index("N/A"),inplace=True)

In [22]:
GP_dict=[]
for row in df.iterrows():
    temp_dict={"Practice Name":row[1][0],
    "Latitude":row[1][1],
    "Longitude":row[1][2],
    "Asthma Percentage":row[1][3]}
    GP_dict.append(temp_dict)

In [23]:
GP_dict

[{'Practice Name': 'Acocks Green Medical Centre',
  'Latitude': 52.4500714,
  'Longitude': -1.8301786,
  'Asthma Percentage': 6.8472},
 {'Practice Name': 'Alpha Medical Practice',
  'Latitude': 52.4912121,
  'Longitude': -1.8566404,
  'Asthma Percentage': 7.9489},
 {'Practice Name': 'Al-Shafa Medical Centre',
  'Latitude': 52.5056399,
  'Longitude': -1.8921101,
  'Asthma Percentage': 6.545},
 {'Practice Name': 'Alum Rock Medical Centre',
  'Latitude': 52.4898917,
  'Longitude': -1.8507548,
  'Asthma Percentage': 7.4716},
 {'Practice Name': 'Amaanah Medical Practice',
  'Latitude': 52.4907265,
  'Longitude': -1.8521147,
  'Asthma Percentage': 6.4019},
 {'Practice Name': 'Apollo Surgery',
  'Latitude': 52.554437,
  'Longitude': -1.8924416,
  'Asthma Percentage': 5.4482},
 {'Practice Name': 'Arden Medical Centre',
  'Latitude': 52.37942899999999,
  'Longitude': -1.7431408,
  'Asthma Percentage': 5.6651},
 {'Practice Name': 'Arran Medical Centre',
  'Latitude': 52.5032514,
  'Longitude': -

In [24]:
df

,GP Practice,Latitude,Longitude,Percentage
0,Acocks Green Medical Centre,52.450071,-1.830179,6.8472
1,Alpha Medical Practice,52.491212,-1.85664,7.9489
2,Al-Shafa Medical Centre,52.50564,-1.89211,6.5450
3,Alum Rock Medical Centre,52.489892,-1.850755,7.4716
4,Amaanah Medical Practice,52.490727,-1.852115,6.4019
...,...,...,...,...
155,Woodland Road Surgery,52.410585,-1.961243,7.2904
156,Wychall Lane Surgery,52.410103,-1.935609,7.4217
157,Yardley Green Medical Centre,52.475873,-1.83905,7.8828
158,Yardley Medical Centre,52.463084,-1.828371,4.0701


In [37]:
Weather_API=OWM

In [107]:
aqi=[]
co=[]
no=[]
no2=[]
o3=[]
so2=[]
pm2_5=[]
pm10=[]
nh3=[]

weather_url="http://api.openweathermap.org/data/2.5/air_pollution/history?"

for row in df.iterrows():
    
    params={"appid":Weather_API,
            'lat':row[1][1],
            'lon':row[1][2],
            'start':"1606266000",
            'end':"1637798400"
            }
    
    temp = requests.get(weather_url, params=params).json()
    
    avg1=0
    avg2=0
    avg3=0
    avg4=0
    avg5=0
    avg6=0
    avg7=0
    avg8=0
    avg9=0
    count=0
    for i in temp['list']:
        count+=1
        avg1=avg1+int(i['main']['aqi'])
        avg2=avg2+int(i['components']['co'])
        avg3=avg3+int(i['components']['no'])
        avg4=avg4+int(i['components']['no2'])
        avg5=avg5+int(i['components']['o3'])
        avg6=avg6+int(i['components']['so2'])
        avg7=avg7+int(i['components']['pm2_5'])
        avg8=avg8+int(i['components']['pm10'])
        avg9=avg9+int(i['components']['nh3'])
    
    aqi.append(avg1/count)
    co.append(avg2/count)
    no.append(avg3/count)
    no2.append(avg4/count)
    o3.append(avg5/count)
    so2.append(avg6/count)
    pm2_5.append(avg7/count)
    pm10.append(avg8/count)
    nh3.append(avg9/count)

In [110]:
df['aqi']=aqi
df['co']=co
df['no']=no
df['no2']=no2
df['o3']=o3
df['so2']=so2
df['pm2_5']=pm2_5
df['pm10']=pm10
df['nh3']=nh3

In [113]:
GP_dict=[]
for row in df.iterrows():
    temp_dict={"Practice Name":row[1][0],
    "Latitude":row[1][1],
    "Longitude":row[1][2],
    "Asthma Percentage":row[1][3],
    "aqi":row[1][4],
    "co":row[1][5],
    "no":row[1][6],
    "no2":row[1][7],
    "o3":row[1][8],
    "so2":row[1][9],
    "pm2_5":row[1][10],
    "pm10":row[1][11],
    "nh3":row[1][12]
              }
    GP_dict.append(temp_dict)

In [114]:
GP_dict

[{'Practice Name': 'Acocks Green Medical Centre',
  'Latitude': 52.4500714,
  'Longitude': -1.8301786,
  'Asthma Percentage': 6.8472,
  'aqi': 1.308379120879121,
  'co': 234.77861721611723,
  'no': 1.8444368131868132,
  'no2': 13.209478021978022,
  'o3': 49.76087454212454,
  'so2': 3.203525641025641,
  'pm2_5': 4.426282051282051,
  'pm10': 5.756181318681318,
  'nh3': 1.3494734432234432},
 {'Practice Name': 'Alpha Medical Practice',
  'Latitude': 52.4912121,
  'Longitude': -1.8566404,
  'Asthma Percentage': 7.9489,
  'aqi': 1.308379120879121,
  'co': 234.77861721611723,
  'no': 1.8444368131868132,
  'no2': 13.209478021978022,
  'o3': 49.76087454212454,
  'so2': 3.203525641025641,
  'pm2_5': 4.426282051282051,
  'pm10': 5.756181318681318,
  'nh3': 1.3494734432234432},
 {'Practice Name': 'Al-Shafa Medical Centre',
  'Latitude': 52.5056399,
  'Longitude': -1.8921101,
  'Asthma Percentage': 6.545,
  'aqi': 1.31856684981685,
  'co': 235.79246794871796,
  'no': 2.2152014652014653,
  'no2': 12

In [115]:
df

,GP Practice,Latitude,Longitude,Percentage,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,Acocks Green Medical Centre,52.450071,-1.830179,6.8472,1.308379,234.778617,1.844437,13.209478,49.760875,3.203526,4.426282,5.756181,1.349473
1,Alpha Medical Practice,52.491212,-1.85664,7.9489,1.308379,234.778617,1.844437,13.209478,49.760875,3.203526,4.426282,5.756181,1.349473
2,Al-Shafa Medical Centre,52.50564,-1.89211,6.5450,1.318567,235.792468,2.215201,12.828526,51.746223,3.257097,4.289263,5.570627,1.402587
3,Alum Rock Medical Centre,52.489892,-1.850755,7.4716,1.308379,234.778617,1.844437,13.209478,49.760875,3.203526,4.426282,5.756181,1.349473
4,Amaanah Medical Practice,52.490727,-1.852115,6.4019,1.308379,234.778617,1.844437,13.209478,49.760875,3.203526,4.426282,5.756181,1.349473
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Woodland Road Surgery,52.410585,-1.961243,7.2904,1.303342,234.702610,2.070627,12.904075,49.847642,3.252404,4.359203,5.699176,1.397436
156,Wychall Lane Surgery,52.410103,-1.935609,7.4217,1.303342,234.702610,2.070627,12.904075,49.847642,3.252404,4.359203,5.699176,1.397436
157,Yardley Green Medical Centre,52.475873,-1.83905,7.8828,1.308379,234.778617,1.844437,13.209478,49.760875,3.203526,4.426282,5.756181,1.349473
158,Yardley Medical Centre,52.463084,-1.828371,4.0701,1.308379,234.778617,1.844437,13.209478,49.760875,3.203526,4.426282,5.756181,1.349473


In [133]:
GP_practice_df=[]

for row in df.iterrows():
    print(row[0])
    temp_df=[]
    aqi=[]
    co=[]
    no=[]
    no2=[]
    o3=[]
    so2=[]
    pm2_5=[]
    pm10=[]
    nh3=[]
    dt=[]

    params={"appid":Weather_API,
            'lat':row[1][1],
            'lon':row[1][2],
            'start':"1606266000",
            'end':"1637798400"
            }
    
    temp = requests.get(weather_url, params=params).json()
    
    for i in temp['list']:
        aqi.append(i['main']['aqi'])
        co.append(i['components']['co'])
        no.append(i['components']['no'])
        no2.append(i['components']['no2'])
        o3.append(i['components']['o3'])
        so2.append(i['components']['so2'])
        pm2_5.append(i['components']['pm2_5'])
        pm10.append(i['components']['pm10'])
        nh3.append(i['components']['nh3'])
        dt.append(i['dt'])
        
    temp_df=pd.DataFrame(zip(dt,aqi,co,no,no2,o3,so2,nh3,pm2_5,pm10),columns=["Date(UNIX)","Air Quality Index","Carbon Monoxide","Nitrogen Monoxide","Nitrogen Dioxide","Ozone","Sulphur Dioxide","Ammonia","Particultes(pm2.5)","Particulates(pm10)"])
    
    temp_df.set_index("Date(UNIX)",inplace=True)
    
    GP_practice_df.append(temp_df)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159


In [117]:
row[1]

GP Practice    Yardley Wood Health Centre
Latitude                        52.418981
Longitude                       -1.859604
Percentage                         8.8968
aqi                              1.308379
co                             234.778617
no                               1.844437
no2                             13.209478
o3                              49.760875
so2                              3.203526
pm2_5                            4.426282
pm10                             5.756181
nh3                              1.349473
Name: 159, dtype: object

In [127]:
temp['list'][0]['dt']

1606266000

In [ ]:
temp_df=pd.DataFrame(zip(dt,aqi,co,no,no2,o3,so2,nh3,pm2_5,pm10),columns=["Date(UNIX)","Air Quality Index","Carbon Monoxide","Nitrogen Monoxide","Nitrogen Dioxide","Ozone","Sulphur Dioxide","Ammonia","Particultes(pm2.5)","Particulates(pm10)"])





In [130]:
df["GP Practice"]

0       Acocks Green Medical Centre
1            Alpha Medical Practice
2           Al-Shafa Medical Centre
3          Alum Rock Medical Centre
4          Amaanah Medical Practice
                   ...             
155           Woodland Road Surgery
156            Wychall Lane Surgery
157    Yardley Green Medical Centre
158          Yardley Medical Centre
159      Yardley Wood Health Centre
Name: GP Practice, Length: 159, dtype: object

In [137]:
GP_practice_df[0].set_index("Date(UNIX)",inplace=True)